In [1]:
using Statistics
using Plots
using Conda
using PyCall
using Distributions
using LinearAlgebra

# Conda.add("minepy")

In [2]:
function mysubplot(x, y, xlim=(-4.0, 4.0), ylim=(-4.0, 4.0))
    r = round(Statistics.cor(x, y), digits=1)
    minepy = pyimport("minepy")
    mine = minepy.MINE(alpha=0.6, c=15, est="mic_approx")
    mine.compute_score(x, y)
    mic = round(mine.mic(), digits=1)
    return scatter(x, y,
        title="Pearson r=$r\nMIC=$mic",
        grid=false,
        showaxis=false,
        legend=false,
        xlim=xlim,
        ylim=ylim,
#         color=:dodgerblue,
        titlefontsize=10
    )
end


function rotation(xy, t)
    b = [cos(t)  -sin(t)
         sin(t)   cos(t)]
    return xy * b
end


function mvnormal(n=1000)
    cors = [0.9999, 0.8, 0.4, 0.0, -0.4, -0.8, -0.9999]
    plots = []
    for c in enumerate(cors)
        i = c[1]
        cor = c[2]
        cov = [1  cor
                cor 1]
        d = MvNormal([0;0], cov)
        xy = rand(d, n)'
        push!(plots, mysubplot(xy[:, 1], xy[:, 2]))
    end
    return plots
end


function rotnormal(n=1000)
    ts = [0, pi / 12, pi / 6, pi / 4, pi / 2 - pi / 6, pi / 2 - pi / 12, pi / 2]
    cov = [1   0.9999
           0.9999   1]
    d = MvNormal([0;0], cov)
    xy = rand(d, n)'
    plots = []
    for c in enumerate(ts)
        i = c[1]
        t = c[2]
        xy_r = rotation(xy, t)
        push!(plots, mysubplot(xy_r[:, 1], xy_r[:, 2]))
    end
    return plots
end    
    

function others(n=1000)
    plots = []
    
    d = Uniform(-1, 1)
    x = rand(d, n)'
    y = 4 * ((x.^ 2).- 0.5).^ 2 + rand(d, n)' / 3
    push!(plots, mysubplot(x[1, :], y[1, :], (-1, 1), (-1 / 3, 1 + 1 / 3)))
    
    y = rand(d, n)'
    xy = [x' y']
    xy = rotation(xy, -pi / 8)
    lim = sqrt(2 + sqrt(2)) / sqrt(2)
    push!(plots, mysubplot(xy[:, 1], xy[:, 2], (-lim, lim), (-lim, lim)))
    
    xy = rotation(xy, -pi / 8)
    lim = sqrt(2)
    push!(plots, mysubplot(xy[:, 1], xy[:, 2], (-lim, lim), (-lim, lim)))
    
    y = 2 * x.^2 + rand(d, n)'
    push!(plots, mysubplot(x[1, :], y[1, :], (-1, 1), (-1, 3)))
    
    
    d2 = Uniform(0, 0.5)
    s = rand(d, n)
    s = round.(s)
    s = map(i -> begin
        if i == 1
            return 1
        end
        return -1
    end, s)
    y = (x.^2 + rand(d2, n)').* s'
    push!(plots, mysubplot(x[1, :], y[1, :], (-1.5, 1.5), (-1.5, 1.5)))
    
    d3 = Uniform(0, 1 / 8)
    y = cos.(x * pi) .+ rand(d3, n)
    x = sin.(x * pi) .+ rand(d3, n)
    push!(plots, mysubplot(x[1, :], y[1, :], (-1.5, 1.5), (-1.5, 1.5)))
    
    
    sd = [1 0
      0 1]
    d1 = MvNormal([3;3], sd)
    d2 = MvNormal([-3;3], sd)
    d3 = MvNormal([-3;-3], sd)
    d4 = MvNormal([3;-3], sd)
    xy1 = rand(d1, Int(n / 4))'
    xy2 = rand(d2, Int(n / 4))'
    xy3 = rand(d3, Int(n / 4))'
    xy4 = rand(d4, Int(n / 4))'
    xy = vcat(xy1, xy2, xy3, xy4)
    push!(plots, mysubplot(xy[:, 1], xy[:, 2], (-7, 7), (-7, 7)))
end

others (generic function with 2 methods)

In [3]:
import Random
Random.seed!(0)

p1 = mvnormal(800)
p2 = rotnormal(200)
p3 = others(800)
plot(p1..., p2..., p3..., layout=(3, 7), size=(1200, 500))
savefig("MIC-2d-correlation.pdf")
plot!()